In [1]:
import tensorflow as tf
import cv2

import numpy as np
import os

/home/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_image(img_path):
    image = cv2.imread(img_path)
    image = np.expand_dims(image,axis=0)
    return image

In [3]:
def draw_bounding_box(img,detections,boxes,classes):
    img_height,img_width,_ = img.shape
    
    for i in detections:
        #Draw bounding box 
        
        #Convert points into different format
        ymin, xmin, ymax, xmax = boxes[i]
        
        xmin = int(xmin*img_width)
        xmax = int(xmax*img_width)
        ymin = int(ymin*img_height)
        ymax = int(ymax*img_height)
        
        
        #print("x goes from {0} to {1}".format(xmin,xmax))
        #print("y goes from {0} to {1}".format(ymin,ymax))
        
        
        cv2.rectangle(img,(xmin,ymin),(xmax,ymax),(0,255,0),thickness = 2)
        
        cv2.imwrite("out.jpg",img)

In [4]:

path_to_infer_graph = "model/frozen_inference_graph.pb"

In [5]:
test_imgs_dir = 'Test_Images'

In [6]:
detection_graph = tf.Graph()

In [7]:
img_paths = [os.path.join(test_imgs_dir,fname) for fname in os.listdir(test_imgs_dir) if fname.lower().endswith(".png") or fname.lower().endswith(".jpg")]

In [8]:
img_paths.sort()

In [9]:
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_infer_graph,mode='rb') as graph_file:
        serialized_graph = graph_file.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def)

In [10]:
sess = tf.InteractiveSession(graph = detection_graph)

In [11]:
# names = [n.name for n in sess.graph.as_graph_def().node]

In [12]:
for img_path in img_paths:
    test_img = load_image(img_path)
    
    input_ = sess.graph.get_tensor_by_name("import/image_tensor:0")
    boxes = sess.graph.get_tensor_by_name("import/detection_boxes:0")
    scores = sess.graph.get_tensor_by_name("import/detection_scores:0")
    classes = sess.graph.get_tensor_by_name("import/detection_classes:0")
    num_detections = sess.graph.get_tensor_by_name("import/num_detections:0")
    (boxes, scores, classes, num_detections) = sess.run( 
                    [boxes, scores, classes, num_detections],
                    feed_dict={input_: test_img})
    
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)
    classes = np.squeeze(classes,axis=0)
    test_img = np.squeeze(test_img,axis=0)
    
    detections = []
    for i,score in enumerate(scores):
        if score >=0.20:
            detections.append(i)
    
    draw_bounding_box(test_img,detections,boxes)
        
    #print(boxes,scores,classes,num_detections)